In [ ]:
from edgar_functions import *
from headers import headers

ticker ="MSFT"

In [ ]:
label_dict = get_label_dictionary(ticker, headers)
#pd.DataFrame(label_dict.keys()).value_counts().to_csv('labels.csv')
label_dict

In [ ]:
accn = get_filtered_filings(ticker, ten_k=True, just_accession_numbers=False, headers=headers)
accn

In [ ]:
acc_num = accn["accessionNumber"].iloc[1].replace("-", "")
soup = get_statement_soup( ticker,acc_num,'income_statement', headers=headers,statement_keys_map=statement_keys_map)
statement = process_one_statement(ticker, acc_num, 'income_statement')

In [ ]:
df = rename_statement(statement, label_dict)
df

In [ ]:
#"balance sheet"
acc_num = accn["accessionNumber"].iloc[0].replace("-", "")
soup = get_statement_soup(
    ticker,
    acc_num,
   "balance_sheet",
    headers=headers,
    statement_keys_map=statement_keys_map,
)
statement = process_one_statement(ticker, acc_num, "balance_sheet")

In [ ]:
bsdaf = rename_statement(statement, label_dict)
bsdaf.T

In [ ]:
def get_facts(ticker, headers=headers):
    cik = cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts
get_facts('HA')

In [ ]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")
cik_matching_ticker('HA', headers=headers)

In [ ]:
def get_statement_file_names_in_filing_summary(
    ticker, accession_number, headers=headers
):
    try:
        session = requests.Session()
        cik = cik_matching_ticker(ticker)
        base_link = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}"
        filing_summary_link = f"{base_link}/FilingSummary.xml"
        filing_summary_response = session.get(
            filing_summary_link, headers=headers
        ).content.decode("utf-8")

        filing_summary_soup = BeautifulSoup(filing_summary_response, "lxml-xml")
        statement_file_names_dict = {}

        for report in filing_summary_soup.find_all("Report"):
            file_name = _get_file_name(report)
            short_name, long_name = report.find("ShortName"), report.find("LongName")

            if _is_statement_file(short_name, long_name, file_name):
                statement_file_names_dict[short_name.text.lower()] = file_name

        return statement_file_names_dict

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return {}

In [ ]:
import pandas as pd

# Assuming the JSON file is named "data.json" and is in the current directory


# Read the JSON file into a DataFrame
df = pd.read_json("CIK0001099219.json")

# Display the DataFrame
print(df)


In [ ]:
df['facts']['us-gaap']